In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.calibration import CalibratedClassifierCV

In [2]:
data=pd.read_csv('bank-additional-full.csv',delimiter=';')
data.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [3]:
X=data.drop('y',axis=1)
Y=data['y']

## Data Preprocessing

In [4]:
# Categorical boolean mask
categorical_feature_mask = X.dtypes==object

# filter categorical columns using mask and turn it into a list
categorical_cols = X.columns[categorical_feature_mask].tolist()

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

def add_onehot_to_dataframe(sparse, df, vectorizer, name):
  '''
      This function will add the one hot encoded to the dataframe.

  '''
  for i, col in enumerate(vectorizer.get_feature_names()):
    colname = name+"_"+col
    # df[colname] = pd.SparseSeries(sparse[:, i].toarray().flatten(), fill_value=0)
    df[colname] = sparse[:, i].toarray().ravel().tolist()
  
  return df

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

def add_onehot_to_dataframe(sparse, df, vectorizer, name):
  '''
      This function will add the one hot encoded to the dataframe.

  '''
  for i, col in enumerate(vectorizer.get_feature_names()):
    colname = name+"_"+col
    # df[colname] = pd.SparseSeries(sparse[:, i].toarray().flatten(), fill_value=0)
    df[colname] = sparse[:, i].toarray().ravel().tolist()
  
  return df

def OneHotEncoder(categorical_cols, X_train):
  '''
    This function takes categorical column names as inputs. The objective
    of this function is to take the column names iteratively and encode the 
    features using One hot Encoding mechanism and also adding the encoded feature
    to the respective dataframe.

    The include_cv parameter indicates whether we should include CV dataset or not.
    This is added specifically because when using GridSearchCV or RandomizedSearchCV,
    we only split the dataset into train and test to give more data to training purposes.
    This is done because GridSearchCV splits the data internally anyway.
  '''

  for i in categorical_cols:
    Vectorizer = CountVectorizer(token_pattern="[A-Za-z0-9-.]+")
    print("Encoding for feature: ", i)
    # Encoding training dataset 
    temp_cols = Vectorizer.fit_transform(X_train[i])
    X_train = add_onehot_to_dataframe(temp_cols, X_train, Vectorizer, i)

   

In [35]:
cols_input=list(X_train.columns)
pickle.dump(cols_input, open('cols_input.sav', 'wb'))

In [22]:
#Any missing values were filled with the mean value

def fill_null(df, df_mean, col2use):
    # This function fills the missing values

    # check the columns are present
    for c in col2use:
        assert c in df.columns, c + ' not in df'
        assert c in df_mean.col.values, c+ 'not in df_mean'
    
    # replace the mean 
    for c in col2use:
        mean_value = df_mean.loc[df_mean.col == c,'mean_val'].values[0]
        df[c] = df[c].fillna(mean_value)
    return df

In [23]:
df_mean = X_train[cols_input].mean(axis = 0)
# save the means
df_mean.to_csv('df_mean.csv',index=True)

In [24]:
df_mean_input = pd.read_csv('df_mean.csv', names =['col','mean_val'])
df_mean_input

,col,mean_val
0,NaN,0.000000
1,age,39.987037
2,campaign,2.570553
3,pdays,962.040437
4,previous,0.173467
...,...,...
58,day_of_week_tue,0.196782
59,day_of_week_wed,0.198512
60,poutcome_failure,0.102914
61,poutcome_nonexistent,0.863236


In [37]:
from sklearn.preprocessing import StandardScaler

scaler  = StandardScaler()
scalerfile = 'scaler.sav'
pickle.dump(scaler, open(scalerfile, 'wb'))

In [38]:
# loading it back
scaler = pickle.load(open(scalerfile, 'rb'))

In [25]:
def cleaning(data):
    data_dup = data[data.duplicated(keep="last")]
    data_wd = data.drop_duplicates()
    Y_clean=data_wd['y'].map({'yes':1,'no':0})
    X_clean=data_wd.drop(['y','duration'],axis=1)
    OneHotEncoder(categorical_cols, X_clean)
    # Drop the categorical features as the one hot encoded representation is present
    X_clean = X_clean.drop(categorical_cols, axis=1)
    fill_null(X_clean,df_mean_input, cols_input)
    return data_wd,data_dup,X_clean,Y_clean

In [26]:
dn,df,X_trans,Y_trans=cleaning(data)
df
X_trans

Encoding for feature:  job
Encoding for feature:  marital
Encoding for feature:  education
Encoding for feature:  default
Encoding for feature:  housing
Encoding for feature:  loan
Encoding for feature:  contact
Encoding for feature:  month
Encoding for feature:  day_of_week
Encoding for feature:  poutcome


,age,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,job_admin.,...,month_oct,month_sep,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success
0,56,1,999,0,1.1,93.994,-36.4,4.857,5191.0,0,...,0,0,0,1,0,0,0,0,1,0
1,57,1,999,0,1.1,93.994,-36.4,4.857,5191.0,0,...,0,0,0,1,0,0,0,0,1,0
2,37,1,999,0,1.1,93.994,-36.4,4.857,5191.0,0,...,0,0,0,1,0,0,0,0,1,0
3,40,1,999,0,1.1,93.994,-36.4,4.857,5191.0,1,...,0,0,0,1,0,0,0,0,1,0
4,56,1,999,0,1.1,93.994,-36.4,4.857,5191.0,0,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,1,999,0,-1.1,94.767,-50.8,1.028,4963.6,0,...,0,0,1,0,0,0,0,0,1,0
41184,46,1,999,0,-1.1,94.767,-50.8,1.028,4963.6,0,...,0,0,1,0,0,0,0,0,1,0
41185,56,2,999,0,-1.1,94.767,-50.8,1.028,4963.6,0,...,0,0,1,0,0,0,0,0,1,0
41186,44,1,999,0,-1.1,94.767,-50.8,1.028,4963.6,0,...,0,0,1,0,0,0,0,0,1,0


In [27]:
Y_trans.head()

0    0
1    0
2    0
3    0
4    0
Name: y, dtype: int64

## Train,test split

In [28]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_trans, Y_trans, test_size=0.2)

In [29]:
X_train_scaled=scaler.transform(X_train)
X_test_scaled=scaler.transform(X_test)
pd.DataFrame(X_test_scaled).head()

,0,1,2,3,4,5,6,7,8,9,...,52,53,54,55,56,57,58,59,60,61
0,-0.767856,-0.565407,0.195948,-0.349238,-0.114737,-0.648939,-0.321468,0.288832,0.399254,-0.581370,...,-0.132105,-0.119922,-0.484723,-0.510651,-0.515514,-0.492636,2.014728,-0.338538,0.39783,-0.18709
1,-0.385113,-0.565407,0.195948,-0.349238,-1.196631,-1.178132,-1.228699,-1.328576,-0.938191,-0.581370,...,-0.132105,-0.119922,2.063033,-0.510651,-0.515514,-0.492636,-0.496345,-0.338538,0.39783,-0.18709
2,0.093315,1.236046,0.195948,-0.349238,0.648953,0.719721,0.888172,0.712919,0.332866,-0.581370,...,-0.132105,-0.119922,-0.484723,-0.510651,-0.515514,2.029895,-0.496345,-0.338538,0.39783,-0.18709
3,0.763115,-0.205116,0.195948,-0.349238,0.839875,-0.228344,0.952974,0.775726,0.845991,1.720075,...,-0.132105,-0.119922,-0.484723,-0.510651,-0.515514,-0.492636,2.014728,-0.338538,0.39783,-0.18709
4,-1.629027,0.515465,0.195948,-0.349238,0.839875,0.588716,-0.472674,0.773997,0.845991,-0.581370,...,-0.132105,-0.119922,-0.484723,1.958284,-0.515514,-0.492636,-0.496345,-0.338538,0.39783,-0.18709


## Model fitting

In [30]:
model_LR=LogisticRegression(penalty='l2',C=100,class_weight='balanced')
model_LR.fit(X_train_scaled, y_train)


LogisticRegression(C=100, class_weight='balanced')

In [31]:
model_SVM=SVC(C=0.01,gamma=0.001,kernel="linear")
model_SVM.fit(X_train_scaled,y_train)

SVC(C=0.01, gamma=0.001, kernel='linear')

In [32]:
model_RF=RandomForestClassifier(class_weight='balanced_subsample', max_depth=8,
                       max_features='sqrt', n_estimators=1000, n_jobs=-1,
                       random_state=42)
model_RF.fit(X_train_scaled, y_train)



RandomForestClassifier(class_weight='balanced_subsample', max_depth=8,
                       max_features='sqrt', n_estimators=1000, n_jobs=-1,
                       random_state=42)

## Loading the models in Disk

In [39]:
filename1= 'finalized_model_LR.sav'
pickle.dump(model_LR, open(filename1, 'wb'))
filename2 = 'finalized_model_SVM.sav'
pickle.dump(model_SVM, open(filename2, 'wb'))
filename3 = 'finalized_model_RF.sav'
pickle.dump(model_RF, open(filename3, 'wb'))
cols_input = pickle.load(open('cols_input.sav','rb'))
df_mean_in = pd.read_csv('df_mean.csv', names =['col','mean_val'])
scaler = pickle.load(open('scaler.sav', 'rb'))

## Reloading the models From Disk

In [40]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, roc_auc_score, roc_curve, f1_score

In [41]:

LR_reloaded = pickle.load(open(filename1, 'rb'))

LR_pred=LR_reloaded.predict(X_test_scaled) 

#metrics
precision_final=precision_score(LR_pred,y_test)
recall_final=recall_score(LR_pred,y_test)
f1_final=f1_score(LR_pred,y_test)
auc=roc_auc_score(y_test, LR_pred)
table_final=pd.DataFrame({'Metrics':["Precision","Recall","F1 Score","AUC"],'Score':[precision_final,recall_final,f1_final,auc]})
table_final


,Metrics,Score
0,Precision,0.587845
1,Recall,0.326781
2,F1 Score,0.420055
3,AUC,0.719172


In [42]:
SVM_reloaded = pickle.load(open(filename2, 'rb'))

SVM_pred=SVM_reloaded.predict(X_test_scaled) 

#metrics
precision_final=precision_score(SVM_pred,y_test)
recall_final=recall_score(SVM_pred,y_test)
f1_final=f1_score(SVM_pred,y_test)
auc=roc_auc_score(y_test, SVM_pred)
table_final=pd.DataFrame({'Metrics':["Precision","Recall","F1 Score","AUC"],'Score':[precision_final,recall_final,f1_final,auc]})
table_final

,Metrics,Score
0,Precision,0.166851
1,Recall,0.583012
2,F1 Score,0.259450
3,AUC,0.576059


In [43]:
RF_reloaded = pickle.load(open(filename3, 'rb'))

RF_pred=RF_reloaded.predict(X_test_scaled) 

#metrics
precision_final=precision_score(RF_pred,y_test)
recall_final=recall_score(RF_pred,y_test)
f1_final=f1_score(RF_pred,y_test)
auc=roc_auc_score(y_test, RF_pred)
table_final=pd.DataFrame({'Metrics':["Precision","Recall","F1 Score","AUC"],'Score':[precision_final,recall_final,f1_final,auc]})
table_final

,Metrics,Score
0,Precision,0.575691
1,Recall,0.351552
2,F1 Score,0.436531
3,AUC,0.722302


In [44]:
def proficiency(mod,X_test):
    a,b=0,0
    pred=mod.predict(X_test)
    n=len(X_test)
    for i in np.arange(n):
        if pred[i]==1:
            a+=1
        else:
            b+=1
    print("Number of customers subscribed {}".format(a))
    print("Number of customers who did not subscribed {}".format(b))

In [45]:
proficiency(LR_reloaded,X_test_scaled)

Number of customers subscribed 1628
Number of customers who did not subscribed 6608


In [46]:
proficiency(SVM_reloaded,X_test_scaled)

Number of customers subscribed 259
Number of customers who did not subscribed 7977


In [47]:
proficiency(RF_reloaded,X_test_scaled)

Number of customers subscribed 1482
Number of customers who did not subscribed 6754


## Sample test

In [48]:
choice=pd.DataFrame(X_test_scaled).sample(50)

In [49]:
proficiency(LR_reloaded,choice)

Number of customers subscribed 8
Number of customers who did not subscribed 42


In [50]:
proficiency(SVM_reloaded,choice)

Number of customers subscribed 0
Number of customers who did not subscribed 50


In [51]:
proficiency(RF_reloaded,choice)

Number of customers subscribed 6
Number of customers who did not subscribed 44
